In [2]:
from pathlib import Path 
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
from collections import OrderedDict
import sys
import os
import seaborn as sns
import researchpy as rp
import statsmodels.formula.api as smf
import scipy.stats as stats
from sklearn.preprocessing import StandardScaler
from numpy.linalg import LinAlgError

#sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

In [3]:
if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    derived_data  = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    derived_data  = '/Users/alina/Desktop/MIT/code/data/output/derived_data'
#os.listdir(data_root)


In [14]:
baseline_var = ['src_subject_id', 'interview_date', 'interview_age', 'sex', 'site', 'days_baseline']
dtypes_baseline = { 'src_subject_id' : 'str',
                    'interview_date': 'str' , 
                    'interview_age' : 'int64' ,
                    'sex' : 'str', 
                    'site' : 'int64' ,
                    'days_baseline':  'int64',
                    'version_form': 'str'}
version_form = ['version_form']

qsts = ['snap', 'ssrs',  'pc', 'masc', 'wechsler'] #masc to many missing data 

In [29]:
#outcome variablles 
snap_vars = ['snainatx','snahix' , 'snaoddx'] #inattention_mean, hyperactie mean'snahypax'
ssrs_vars = ['sspintx', 'ssptossx']# social skills mean, internalizing mean 
masc_vars = ['masc_masctotalt']
pc_vars = ['pcrcpax', 'pcrcprx'] # power assertion, personal closeness
wechsler_vars = ['w1readb','w2math','w3spell' ]
outcomes_dict  = {'snap' : snap_vars, 'ssrs' : ssrs_vars,  'masc': masc_vars,'pc': pc_vars, 'wechsler': wechsler_vars}


In [17]:
# load files, drop rows if missing date, drop duplicates 

snap_file = 'snap01.txt'
ssrs_file = 'ssrs01.txt'
masc_file = 'masc_p01.txt'
pc_file = 'pcrc01.txt'
wechsler_file = 'wiat_iiip201.txt'
treat_group_file = 'treatment_groups.csv'
treat_group = pd.read_csv(Path(derived_data, treat_group_file))

In [21]:
snap = prep.get_data(Path(data_root, snap_file), columns= [baseline_var, snap_vars], set_dtypes=True, treat_group=treat_group, version_form=True)
ssrs = prep.get_data(Path(data_root, ssrs_file), columns= [baseline_var, ssrs_vars], set_dtypes=True, treat_group=treat_group, version_form=True)
masc = prep.get_data(Path(data_root, masc_file), columns= [baseline_var, masc_vars], set_dtypes=True, treat_group=treat_group)
pc = prep.get_data(Path(data_root, pc_file), columns= [baseline_var, pc_vars], set_dtypes=True, treat_group=treat_group)
wechsler= prep.get_data(Path(data_root, wechsler_file), columns= [baseline_var, wechsler_vars], set_dtypes=True, treat_group=treat_group)

qsts = ['snap', 'ssrs', 'masc', 'pc', 'wechsler']


Success
Success
Success
Success
Success


In [129]:
ssrs.loc[ssrs['version_form'].str.startswith('Teacher'), 'version_form'] = 'Teacher'
ssrs.loc[ssrs['version_form'].str.startswith('Parent'), 'version_form'] = 'Parent'
data_dict = dict(zip(qsts, [snap, ssrs, masc, pc, wechsler]))
for qst in qsts:
    data_dict[qst] = data_dict[qst][data_dict[qst]['trtname'] != 'L']
    
    # Optionally, remove 'L' from the category list if 'trtname' is categorical
    if data_dict[qst]['trtname'].dtype.name == 'category':
        data_dict[qst]['trtname'] = data_dict[qst]['trtname'].cat.remove_categories(['L'])

# Verify the changes
for qst in qsts[:2]:
    print(data_dict[qst].trtname.unique())

['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['M', 'A', 'C', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']


/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_20574/945923732.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_dict[qst]['trtname'] = data_dict[qst]['trtname'].cat.remove_categories(['L'])


In [131]:
for qst in qsts:
    print(data_dict[qst].trtname.unique())

['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['M', 'A', 'C', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']
['C', 'M', 'A', 'P']
Categories (4, object): ['A', 'C', 'M', 'P']


In [32]:
timepoints = [46 , 168, 319, 500]

,src_subject_id,interview_date,interview_age,sex,snainatx,snaoddx,snahix,days_baseline,site,version_form,trtname
0,P1001,1997-06-28,156,M,0.11,0.00,0.22,0,1,Parent,L
1,P1001,1997-06-28,156,M,0.33,0.00,0.00,0,1,Parent,L
2,P1001,1998-10-02,171,M,0.00,0.00,0.00,461,1,Parent,L
3,P1001,1998-10-23,172,M,0.11,0.00,0.00,482,1,Parent,L
4,P1001,2001-11-26,210,M,0.00,0.00,0.00,1612,1,Parent,L
...,...,...,...,...,...,...,...,...,...,...,...
14539,P1868,2002-11-18,154,F,0.22,0.00,0.00,1753,1,Teacher,L
14540,P1868,2002-11-18,154,F,0.22,0.13,0.33,1753,1,Teacher,L
14541,P1868,2004-10-25,178,F,0.11,0.13,0.33,2460,1,Teacher,L
14542,P1868,2004-11-02,178,F,1.00,0.50,1.33,2468,1,Teacher,L


## MIXED LM 

In [132]:
gen_interact_formula = 'C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)' # reapeat with log days 
formulas =  [[' ~ '.join((var, gen_interact_formula)) for var in values] for values in outcomes_dict.values()]
formulas_dict = dict(zip(outcomes_dict.keys(), formulas))
formulas_dict

{'snap': ['snainatx ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)',
  'snahix ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)',
  'snaoddx ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)'],
 'ssrs': ['sspintx ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)',
  'ssptossx ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)'],
 'masc': ['masc_masctotalt ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)'],
 'pc': ['pcrcpax ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = "A")) * C(site)',
  'pcrcprx ~ C(trtname, Treatment(reference = "A")) * days_baseline +  C(trtname, Treatment(reference = 

In [133]:
groups = 'src_subject_id'
alpha = 0.05
data = data_dict['snap'][data_dict['snap']['days_baseline'] < timepoints[2]]
formula = formulas_dict['snap'][1]
raters = ['Teacher', 'Parent']

### Separate per rater

In [134]:
test_result = smf.mixedlm(formula= formula, data= data, groups = groups).fit()

In [138]:
result_snap = []
for rater in raters:
    for i in range(len(snap_vars)):
        data = data_dict['snap'][data_dict['snap']['days_baseline'] < timepoints[1]]
        try : 
            print(rater, snap_vars[i])
            result_snap.append(smf.mixedlm(formulas_dict['snap'][i], data[data['version_form'] == rater], groups = groups).fit() )
        except LinAlgError as e:
            print('Error', rater, snap_vars[i])
            print(data_dict['snap'][data_dict['snap']['version_form'] == rater].shape)
            print(e)
            continue
        
        
        
result_ssrs = []
for rater in raters:
    for i in range(len(ssrs_vars)):
        data = data_dict['ssrs'][data_dict['ssrs']['days_baseline'] < timepoints[1]]
        try : 
            print(rater, ssrs_vars[i])
            result_ssrs.append(smf.mixedlm(formulas_dict['ssrs'][i], data[data['version_form'] == rater], groups = groups).fit() )
        except LinAlgError as e:
            print('Error', rater, snap_vars[i])
            print(data_dict['ssrs'][data_dict['ssrs']['version_form'] == rater].shape)
            print(e)
            continue

Teacher snainatx
Teacher snahix
Teacher snaoddx
Parent snainatx
Parent snahix
Parent snaoddx
Teacher sspintx
Teacher ssptossx
Parent sspintx
Parent ssptossx


In [136]:
for result in result_snap:
    print(result.summary())

                              Mixed Linear Model Regression Results
Model:                          MixedLM               Dependent Variable:               snainatx  
No. Observations:               1459                  Method:                           REML      
No. Groups:                     579                   Scale:                            0.2628    
Min. group size:                1                     Log-Likelihood:                   -1436.1218
Max. group size:                5                     Converged:                        Yes       
Mean group size:                2.5                                                               
--------------------------------------------------------------------------------------------------
                                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------------
Intercept                                

In [137]:
result_pc = []
for i in range(len(pc_vars)):
    data = data_dict['pc'][data_dict['pc']['days_baseline'] < timepoints[1]+ 500]
    try : 
        print(rater, ssrs_vars[i])
        result_pc.append(smf.mixedlm(formulas_dict['pc'][i], data, groups = groups).fit() )
    except LinAlgError as e:
        print('Error', rater, pc_vars[i])
        print(data_dict['pc'].shape)
        print(e)
        continue

Parent sspintx
Parent ssptossx


In [88]:
outcomes_dict

{'snap': ['snainatx', 'snahix', 'snaoddx'],
 'ssrs': ['sspintx', 'ssptossx'],
 'masc': ['masc_masctotalt'],
 'pc': ['pcrcpax', 'pcrcprx'],
 'wechsler': ['w1readb', 'w2math', 'w3spell']}

In [139]:
time = timepoints[1]
delta = 50
method = 'bfgs'

results_qst = []
results_dict = {}

for qst in qsts:
    if qst in ['ssrs', 'snap']:
        results_dict_r = {}
        
        for rater in raters:
            for i in range(len(outcomes_dict[qst])):
                data = data_dict[qst][data_dict[qst]['days_baseline'] < time]

                # Check that data is not empty after filtering
                if data.empty:
                    print(f"Insufficient data for rater {rater} in questionnaire {qst}")
                    continue
                
                print(data.shape, data[groups].shape, data[groups].nunique())

                try: 
                    print(f"Running for rater {rater}, variable {outcomes_dict[qst][i]}")
                    
                    # Fit the model and specify the optimizer
                    result = smf.mixedlm(formulas_dict[qst][i], 
                                         data[data['version_form'] == rater], 
                                         groups=groups).fit()
                    
                    results_qst.append(result)
                
                except LinAlgError as e:
                    print(f"Linear Algebra Error for rater {rater} and variable {outcomes_dict[qst][i]}")
                    print(data_dict[qst][data_dict[qst]['version_form'] == rater].shape)
                    print(e)
                    continue
            
            # Store results for each rater
            results_dict_r[rater] = results_qst.copy()
            results_qst.clear()  # Clear list to avoid accumulating across raters

        results_dict[qst] = results_dict_r

    else:
        for i in range(len(outcomes_dict[qst])):
            data = data_dict[qst][data_dict[qst]['days_baseline'] < time]
            
            try: 
                print(f"Running for other outcomes, variable {outcomes_dict[qst][i]}")
                
                result = smf.mixedlm(formulas_dict[qst][i], data, groups=data[groups]).fit()
                
                results_qst.append(result)
            
            except LinAlgError as e:
                print(f"Linear Algebra Error for variable {outcomes_dict[qst][i]}")
                print(data_dict[qst].shape)
                print(e)
                continue
        
        results_dict[qst] = results_qst.copy()
        results_qst.clear()  # Clear list to avoid accumulating results across outcomes

            

(3545, 11) (3545,) 579
Running for rater Teacher, variable snainatx
(3545, 11) (3545,) 579
Running for rater Teacher, variable snahix
(3545, 11) (3545,) 579
Running for rater Teacher, variable snaoddx
(3545, 11) (3545,) 579
Running for rater Parent, variable snainatx
(3545, 11) (3545,) 579
Running for rater Parent, variable snahix
(3545, 11) (3545,) 579
Running for rater Parent, variable snaoddx
(2414, 10) (2414,) 579
Running for rater Teacher, variable sspintx
(2414, 10) (2414,) 579
Running for rater Teacher, variable ssptossx
(2414, 10) (2414,) 579
Running for rater Parent, variable sspintx
(2414, 10) (2414,) 579
Running for rater Parent, variable ssptossx
Running for other outcomes, variable masc_masctotalt
Running for other outcomes, variable pcrcpax
Running for other outcomes, variable pcrcprx
Running for other outcomes, variable w1readb
Running for other outcomes, variable w2math


/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/alina/opt/anaconda3/envs/abcd/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)


Running for other outcomes, variable w3spell


In [141]:
results_dict

{'snap': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d6daad0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8e81930>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d460160>]},
 'ssrs': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8ead1e0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae51630>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b4c7580>]},
 'masc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae76770>],
 'pc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae2df00>,
 'wechsler': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd68a7652a0>,
  <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b1308b0>]}

In [154]:
hyps_interaction = var_dict.get_hyps_interactions()

In [155]:
hyps_interaction

{'time_treat': 'C(trtname, Treatment(reference="A"))[T.M]:days_baseline = C(trtname, Treatment(reference="A"))[T.P]:days_baseline  = C(trtname, Treatment(reference="A"))[T.C]:days_baseline = 0',
 'site_treat': 'C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.6] = C(trtname, Treatment(reference="A"))[T.P]:C(site

In [158]:
f_test_snap14_p = rr.f_test_interactions(results_dict['snap']['Teacher'][0], hyps_interaction, alpha)

In [159]:
f_test_snap14_p

,Description,Significance,F-Value,P-Value
0,time_treat,*Significant*,7.114258,0.000096
1,site_treat,Not Significant,1.421438,0.128840
2,site,Not Significant,0.218636,0.954637


In [220]:
results_table = pd.read_csv(Path(derived_data, 'updated_rr_results_table.csv'))

In [165]:
results_dict

{'snap': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d6daad0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8e81930>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d460160>]},
 'ssrs': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8ead1e0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae51630>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b4c7580>]},
 'masc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae76770>],
 'pc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae2df00>,
 'wechsler': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd68a7652a0>,
  <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b1308b0>]}

In [208]:
outcomes_list = []
for out in outcomes_dict.values():
    outcomes_list.append(out)

In [209]:
outcomes_written = [['Inattention', 'Hyperactive', 'ODD'], 
    ['SSRS Internalizing', 'SSRS social skills'], 
    ['MASC'], 
    ['Power assertion', 'Personal'],
    ['eading'], ['Math'], ['Spelling']]
outcomes_written = np.concatenate(outcomes_written)
outcomes_list = np.concatenate(outcomes_list)

In [181]:
outcomes_list

[['snainatx', 'snahix', 'snaoddx'],
 ['sspintx', 'ssptossx'],
 ['masc_masctotalt'],
 ['pcrcpax', 'pcrcprx'],
 ['w1readb', 'w2math', 'w3spell']]

In [210]:
outcomes_written_dict = dict(zip(outcomes_list, outcomes_written))
outcomes_written_dict

{'snainatx': 'Inattention',
 'snahix': 'Hyperactive',
 'snaoddx': 'ODD',
 'sspintx': 'SSRS Internalizing',
 'ssptossx': 'SSRS social skills',
 'masc_masctotalt': 'MASC',
 'pcrcpax': 'Power assertion',
 'pcrcprx': 'Personal',
 'w1readb': 'eading',
 'w2math': 'Math',
 'w3spell': 'Spelling'}

In [187]:
results_dict

{'snap': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d6daad0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8e81930>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd69d460160>]},
 'ssrs': {'Teacher': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd6a8ead1e0>,
  'Parent': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae51630>,
   <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b4c7580>]},
 'masc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae76770>],
 'pc': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66ae2df00>,
 'wechsler': [<statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd68a7652a0>,
  <statsmodels.regression.mixed_linear_model.MixedLMResultsWrapper at 0x7fd66b1308b0>]}

In [190]:
results_dict['snap']['Teacher'][0].summary()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                              Mixed Linear Model Regression Results
==================================================================================================
Model:                          MixedLM               Dependent Variable:               snainatx  
No. Observations:               1371                  Method:                           REML      
No. Groups:                     579                   Scale:                            0.2313    
Min. group size:                1                     Log-Likelihood:                   -1306.5916
Max. group size:                4                     Converged:                        Yes       
Mean group size:                2.4                                                               
--------------------------------------------------------------------------------------------------
                                                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------------
Intercept                                                2.177    0.115 18.873 0.000  1.951  2.403
C(trtname, Treatment(reference="A"))[T.C]               -0.142    0.162 -0.877 0.381 -0.459  0.175
C(trtname, Treatment(reference="A"))[T.M]               -0.163    0.164 -0.994 0.320 -0.486  0.159
C(trtname, Treatment(reference="A"))[T.P]               -0.042    0.162 -0.256 0.798 -0.360  0.277
C(site)[T.2]                                             0.074    0.165  0.451 0.652 -0.249  0.398
C(site)[T.3]                                            -0.047    0.164 -0.285 0.776 -0.367  0.274
C(site)[T.4]                                            -0.074    0.159 -0.465 0.642 -0.386  0.238
C(site)[T.5]                                            -0.039    0.160 -0.242 0.809 -0.352  0.274
C(site)[T.6]                                             0.019    0.158  0.118 0.906 -0.291  0.328
C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.2]   0.206    0.232  0.887 0.375 -0.249  0.661
C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.2]   0.121    0.235  0.514 0.607 -0.340  0.582
C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.2]   0.221    0.232  0.950 0.342 -0.234  0.676
C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.3]   0.028    0.230  0.120 0.904 -0.422  0.477
C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.3]  -0.177    0.230 -0.767 0.443 -0.628  0.275
C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.3]   0.259    0.230  1.127 0.260 -0.191  0.709
C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.4]  -0.199    0.224 -0.886 0.375 -0.639  0.241
C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.4]   0.253    0.226  1.120 0.263 -0.190  0.696
C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.4]   0.148    0.225  0.655 0.512 -0.294  0.589
C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.5]  -0.267    0.225 -1.183 0.237 -0.708  0.175
C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.5]   0.186    0.227  0.822 0.411 -0.258  0.631
C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.5]   0.038    0.225  0.169 0.865 -0.404  0.480
C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.6]  -0.243    0.222 -1.092 0.275 -0.678  0.193
C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.6]   0.021    0.225  0.091 0.927 -0.421  0.462
C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.6]   0.046    0.223  0.207 0.836 -0.392  0.484
days_baseline                                           -0.003    0.000 -7.039 0.000 -0.004 -0.002
C(trtname, Treatment(reference="A"))[T.C]:days_baseline -0.002    0.001 -3.026 0.002 -0.003 -0.001
C(trtname, Treatment(reference="A"))[T.M]:days_baseline -0.001    0.001 -2.328 0.020 -0.002 -0.000
C(trtname, Treatment(reference="A"))[T.P]:days_baseline  0.000    0.001  0.822 0.411 -0.001  0.002
src_subject_id Var                                       0.203    0.049                           
======================

In [211]:
hypotheses = var_dict.get_hyps_interactions()
hypotheses

{'time_treat': 'C(trtname, Treatment(reference="A"))[T.M]:days_baseline = C(trtname, Treatment(reference="A"))[T.P]:days_baseline  = C(trtname, Treatment(reference="A"))[T.C]:days_baseline = 0',
 'site_treat': 'C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.2] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.3] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.4] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.P]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.C]:C(site)[T.5] = C(trtname, Treatment(reference="A"))[T.M]:C(site)[T.6] = C(trtname, Treatment(reference="A"))[T.P]:C(site

In [212]:
f_test_results = {}

# Iterate over each questionnaire in results_dict
for qst, data in results_dict.items():
    f_test_results[qst] = {}
    
    # Handle 'snap' and 'ssrs' with specific raters
    if qst in ['snap', 'ssrs']:
        for rater in raters:
            f_test_results[qst][rater] = {}
            outcomes = outcomes_dict[qst]  # Get the list of outcomes for this questionnaire

            # Iterate over each outcome and corresponding model in results_dict
            for i, model in enumerate(data[rater]):
                outcome_name = outcomes[i]  # Get the outcome name in order
                f_test_results[qst][rater][outcome_name] = {}

                # Calculate F and p-values for each hypothesis
                for hyp_key, hyp in hypotheses.items():
                    f_test_result = model.f_test(hyp)
                    f_value = f_test_result.fvalue  # Extract scalar F-value
                    p_value = f_test_result.pvalue

                    # Store the F and p-values
                    f_test_results[qst][rater][outcome_name][hyp_key] = {
                        "F_value": f_value,
                        "p_value": p_value
                    }

    else:  # For 'masc', 'pc', 'wechsler' without specific raters (only 'Child')
        outcomes = outcomes_dict[qst]  # Get the list of outcomes for this questionnaire
        
        # Iterate over each outcome and corresponding model in results_dict
        for i, model in enumerate(data):
            outcome_name = outcomes[i]  # Get the outcome name in order
            f_test_results[qst][outcome_name] = {}

            # Calculate F and p-values for each hypothesis
            for hyp_key, hyp in hypotheses.items():
                f_test_result = model.f_test(hyp)
                f_value = f_test_result.fvalue # Extract scalar F-value
                p_value = f_test_result.pvalue

                # Store the F and p-values
                f_test_results[qst][outcome_name][hyp_key] = {
                    "F_value": f_value,
                    "p_value": p_value
                }

# Display the calculated F and p-values for verification
f_test_results

{'snap': {'Teacher': {'snainatx': {'time_treat': {'F_value': 7.11425808574726,
     'p_value': 9.646144332795659e-05},
    'site_treat': {'F_value': 1.42143812628363, 'p_value': 0.1288404298921499},
    'site': {'F_value': 0.2186361107905487, 'p_value': 0.9546372839039173}},
   'snahix': {'time_treat': {'F_value': 4.151559744607934,
     'p_value': 0.006125910589437124},
    'site_treat': {'F_value': 1.0840436796005173,
     'p_value': 0.3662134740654153},
    'site': {'F_value': 2.169635470321894, 'p_value': 0.05513887368251657}},
   'snaoddx': {'time_treat': {'F_value': 7.149311740265465,
     'p_value': 9.179924093171846e-05},
    'site_treat': {'F_value': 0.6988281116359728,
     'p_value': 0.787692875557686},
    'site': {'F_value': 2.1560725277534436, 'p_value': 0.05658343928134424}}},
  'Parent': {'snainatx': {'time_treat': {'F_value': 13.576263966824902,
     'p_value': 8.953252702988073e-09},
    'site_treat': {'F_value': 0.5965912559760242,
     'p_value': 0.8797403372445046}

In [221]:
# Ensure columns for replicated F and P values exist in results_table
if 'replicated F-value' not in results_table.columns:
    results_table['replicated F-value'] = None
if 'replicated P-value' not in results_table.columns:
    results_table['replicated P-value'] = None

# Map hypothesis keys to test names in results_table
test_name_mapping = {
    'time_treat': "Treatment x time",
    'site_treat': "Treatment x site",
    'site': "Site"
}

# Populate results_table with replicated F and P values from f_test_results
for qst, data in f_test_results.items():
    if qst in ['snap', 'ssrs']:  # Handle 'snap' and 'ssrs' with specific raters
        for rater, outcomes in data.items():
            for outcome_name, results in outcomes.items():
                # Get the measure name from outcomes_written_dict for partial match
                measure_name = outcomes_written_dict.get(outcome_name, None)
                if measure_name is None:
                    continue

                # Populate F and p values for each hypothesis
                for hyp_key, test_name in test_name_mapping.items():
                    if hyp_key in results:
                        f_value = results[hyp_key]["F_value"]
                        p_value = results[hyp_key]["p_value"]

                        # Find the row in results_table where Measure contains measure_name and matches rater and test
                        match_row = (
                            results_table['Measure'].str.contains(measure_name, case=False, na=False) &
                            (results_table['Rater'] == rater) &
                            (results_table['Test'] == test_name)
                        )

                        # Insert the replicated F and p values
                        results_table.loc[match_row, 'replicated F-value'] = f_value
                        results_table.loc[match_row, 'replicated P-value'] = p_value

    else:  # Handle 'masc', 'pc', 'wechsler' without specific raters (only 'Child')
        for outcome_name, results in data.items():
            # Get the measure name from outcomes_written_dict for partial match
            measure_name = outcomes_written_dict.get(outcome_name, None)
            if measure_name is None:
                continue

            # Populate F and p values for each hypothesis
            for hyp_key, test_name in test_name_mapping.items():
                if hyp_key in results:
                    f_value = results[hyp_key]["F_value"]
                    p_value = results[hyp_key]["p_value"]

                    # Find the row in results_table where Measure contains measure_name and matches test
                    match_row = (
                        results_table['Measure'].str.contains(measure_name, case=False, na=False) &
                        (results_table['Test'] == test_name)
                    )

                    # Insert the replicated F and p values
                    results_table.loc[match_row, 'replicated F-value'] = f_value
                    results_table.loc[match_row, 'replicated P-value'] = p_value

# Display the updated table for verification
results_table.head(10)

,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,index,replicated F-value,replicated P-value
0,0.0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.6,<.001,NaN,7.114258,0.000096
1,1.0,ADHD symptoms,Inattention,Teacher,Treatment x site,0.9,.56,NaN,1.421438,0.12884
2,2.0,ADHD symptoms,Inattention,Teacher,Site,2.7,.02,NaN,0.218636,0.954637
3,3.0,ADHD symptoms,Inattention,Parent,Treatment x time,21.5,<.001,NaN,13.576264,0.0
4,4.0,ADHD symptoms,Inattention,Parent,Treatment x site,0.6,.88,NaN,0.596591,0.87974
5,5.0,ADHD symptoms,Inattention,Parent,Site,3.0,.02,NaN,0.330446,0.894798
6,6.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.0,<.001,NaN,4.15156,0.006126
7,7.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.3,.49,NaN,1.084044,0.366213
8,8.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.0,.02,NaN,2.169635,0.055139
9,9.0,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.5,<.001,NaN,13.364109,0.0


In [222]:
results_table.reset_index()

,level_0,Unnamed: 0,Outcome Domain,Measure,Rater,Test,F_value,P_value,index,replicated F-value,replicated P-value
0,0,0.0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.60,<.001,NaN,7.114258,0.000096
1,1,1.0,ADHD symptoms,Inattention,Teacher,Treatment x site,0.90,.56,NaN,1.421438,0.12884
2,2,2.0,ADHD symptoms,Inattention,Teacher,Site,2.70,.02,NaN,0.218636,0.954637
3,3,3.0,ADHD symptoms,Inattention,Parent,Treatment x time,21.50,<.001,NaN,13.576264,0.0
4,4,4.0,ADHD symptoms,Inattention,Parent,Treatment x site,0.60,.88,NaN,0.596591,0.87974
5,5,5.0,ADHD symptoms,Inattention,Parent,Site,3.00,.02,NaN,0.330446,0.894798
6,6,6.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.00,<.001,NaN,4.15156,0.006126
7,7,7.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.30,.49,NaN,1.084044,0.366213
8,8,8.0,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.00,.02,NaN,2.169635,0.055139
9,9,9.0,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.50,<.001,NaN,13.364109,0.0


In [1]:
outcomes_dict


NameError: name 'outcomes_dict' is not defined

In [226]:
results_table = results_table[results_table['Measure'] != 'Classroom observer']
def convert_p_value(val):
    # Check if the value contains '<', if so return as is
    if isinstance(val, str) and '<' in val:
        return val
    # Otherwise, try to convert to integer
    try:
        return float(val)  # Convert to float first, then to int
    except (ValueError, TypeError):
        return val  # Return original value if conversion fails

# Apply the conversion function to 'P_value' column
results_table['P_value'] = results_table['P_value'].apply(convert_p_value)


In [230]:


# Function to highlight p-values below 0.05 (including 'p<0.001')
def highlight_p_values_row(row):
    # Check if both 'P_value' and 'replicated P-value' are below 0.05 or contain '<.001'
    p_value = row['P_value']
    replicated_p_value = row['replicated P-value']
    
    # Determine if each cell should be highlighted blue or green
    if ((isinstance(p_value, str) and p_value == '<.001') or (isinstance(p_value, float) and p_value < 0.05)) and \
       ((isinstance(replicated_p_value, str) and replicated_p_value == '<.001') or (isinstance(replicated_p_value, float) and replicated_p_value < 0.05)):
        return ['background-color: green'] * len(row)  # Entire row green if both conditions are met
    else:
        # Apply blue background to individual cells as needed
        return [
            'background-color: blue' if (isinstance(val, str) and val == 'p<0.001') or (isinstance(val, float) and val < 0.05) else ''
            for val in row
        ]



In [233]:
results_table = results_table.drop(columns=['index', 'Unnamed: 0'])

In [234]:

# Apply the highlighting function row-wise
styled_results_table = results_table.style.apply(highlight_p_values_row, axis=1, subset=['P_value', 'replicated P-value'])

# Display the styled table with conditional highlighting
styled_results_table

,Outcome Domain,Measure,Rater,Test,F_value,P_value,replicated F-value,replicated P-value
0,ADHD symptoms,Inattention,Teacher,Treatment x time,10.600000,<.001,7.114258,0.000096
1,ADHD symptoms,Inattention,Teacher,Treatment x site,0.900000,0.560000,1.421438,0.128840
2,ADHD symptoms,Inattention,Teacher,Site,2.700000,0.020000,0.218636,0.954637
3,ADHD symptoms,Inattention,Parent,Treatment x time,21.500000,<.001,13.576264,0.000000
4,ADHD symptoms,Inattention,Parent,Treatment x site,0.600000,0.880000,0.596591,0.879740
5,ADHD symptoms,Inattention,Parent,Site,3.000000,0.020000,0.330446,0.894798
6,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x time,10.000000,<.001,4.151560,0.006126
7,ADHD symptoms,Hyperactive-impulsive,Teacher,Treatment x site,1.300000,0.490000,1.084044,0.366213
8,ADHD symptoms,Hyperactive-impulsive,Teacher,Site,3.000000,0.020000,2.169635,0.055139
9,ADHD symptoms,Hyperactive-impulsive,Parent,Treatment x time,21.500000,<.001,13.364109,0.000000


In [235]:
results_table.to_csv(Path(derived_data, 'results_table_rr_and_new.csv'), index=False)